In [1]:
! pip install transformers

In [2]:
! pip install --upgrade huggingface_hub

In [3]:
!pip install ipywidgets
from ipywidgets import widgets, interactive
from huggingface_hub import notebook_login

In [4]:
# notebook_login()

In [5]:
!pip install datasets

In [6]:
import transformers

print(transformers.__version__)

4.26.1


In [7]:
from transformers.utils import send_example_telemetry

send_example_telemetry("translation_notebook", framework="tf")

In [8]:
model_checkpoint = "Helsinki-NLP/opus-mt-en-ROMANCE"

In [9]:
! pip install datasets transformers[sentencepiece] sacrebleu
! pip install sacrebleu

zsh:1: no matches found: transformers[sentencepiece]


In [10]:
from datasets import load_dataset, load_metric
import json

In [11]:

metric = load_metric("sacrebleu")

raw_dataset = load_dataset('json', data_files={'train': ['content/output.jsonl', 'content/output2flores.jsonl', 'content/output3org.jsonl', 'content/output4med.jsonl', 'content/output5mafand.jsonl', 'content/output6bay.jsonl'], 'validation': 'content/dev.jsonl', 'test': 'content/test.jsonl'})
#excluded '/content/train.jsonl'

/var/folders/vq/qfqzj01x6ms_n2n90qm33bvr0000gn/T/ipykernel_33828/567929508.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")
Using custom data configuration default-1d10f823a78c2103
Found cached dataset json (/Users/michael/.cache/huggingface/datasets/json/default-1d10f823a78c2103/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)


  0%|          | 0/3 [00:00<?, ?it/s]

In [12]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 64341
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 268
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 228
    })
})

In [13]:
raw_dataset["train"][0]

{'translation': {'bam': 'Lilian Diana (Ɔkutɔburukalo tile tan 14, san 1893 – fewuruye tile 27, san 1993) ye Ameriki filimu kela, kuntigi ani siniman glana de kun do.',
  'fr': 'Lillian Diana Gish (14 octobre 1893 - 27 février 1993) était une actrice, réalisatrice et scénariste américaine.'}}

In [14]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [15]:
show_random_elements(raw_dataset["train"])

,translation
0,"{'bam': '6', 'fr': 'Chapitre 6'}"
1,"{'bam': 'Muso ka malo siw sɛnɛta.', 'fr': 'Les variétés locales des femmes, bien meilleures'}"
2,"{'bam': 'U matigi fana bɛ misi di u ma.', 'fr': 'et en échange, leur maître leur donne une vache.'}"
3,"{'bam': 'A ko mun don?', 'fr': 'lle dit : « Qu'est-ce que c'est ?'}"
4,"{'bam': 'Min bɛ u ni san cɛ, mɔgɔw kɔni bɛ u kali o de Tɔgɔ la, ani u ni ɲɔgɔn cɛ la ; o kalili ye ko jɛlen de ye min bɛ sɔsɔli bɛɛ ban.', 'fr': 'Les hommes jurent par un plus grand, et, entre eux, la garantie du serment met un terme à toute contestation.'}"


In [16]:
metric

Metric(name: "sacrebleu", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id='references')}, usage: """
Produces BLEU scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions (`list` of `str`): list of translations to score. Each translation should be tokenized into a list of tokens.
    references (`list` of `list` of `str`): A list of lists of references. The contents of the first sub-list are the references for the first prediction, the contents of the second sub-list are for the second prediction, etc. Note that there must be the same number of references for each prediction (i.e. all sub-lists must be of the same length).
    smooth_method (`str`): The smoothing method to use, defaults to `'exp'`. Possible values are:
        - `'none'`: no smoothing
        - `'floor'`: increment zero counts
        - `'add-k'`: increment num/deno

In [17]:
fake_preds = ["hello there", "general kenobi"]
fake_labels = [["hello there"], ["general kenobi"]]
metric.compute(predictions=fake_preds, references=fake_labels)

{'score': 0.0,
 'counts': [4, 2, 0, 0],
 'totals': [4, 2, 0, 0],
 'precisions': [100.0, 100.0, 0.0, 0.0],
 'bp': 1.0,
 'sys_len': 4,
 'ref_len': 4}

In [18]:
!pip install sentencepiece
!pip install sacremoses
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [19]:
if "mbart" in model_checkpoint:
    tokenizer.src_lang = "bam-XX"
    tokenizer.tgt_lang = "fr-FR"

In [20]:
tokenizer("Hello, this is a sentence!")

{'input_ids': [4708, 2, 69, 28, 9, 8662, 84, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [21]:
tokenizer(["Hello, this is a sentence!", "This is another sentence."])

{'input_ids': [[4708, 2, 69, 28, 9, 8662, 84, 0], [188, 28, 823, 8662, 3, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

In [22]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["Hello, this is a sentence!", "This is another sentence."]))

{'input_ids': [[14232, 244, 2, 69, 160, 6, 9, 10513, 1101, 84, 0], [13486, 6, 160, 6, 3778, 4853, 10513, 1101, 3, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


/Users/michael/miniconda3/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3581: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [23]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "translate English to Romanian: "
else:
    prefix = ""

In [24]:
max_input_length = 128
max_target_length = 128
source_lang = "bam"
target_lang = "fr"


def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [25]:
preprocess_function(raw_dataset["train"][:2])

{'input_ids': [[46157, 440, 42575, 23, 54309, 7027, 47, 52272, 6106, 41139, 244, 41081, 2240, 12513, 5525, 337, 3936, 375, 1129, 19275, 4919, 41081, 20102, 5525, 3271, 39, 8758, 67, 1633, 28361, 6815, 1479, 190, 23218, 242, 2, 2573, 184, 47, 21504, 90, 75, 108, 2630, 677, 21523, 326, 5, 2573, 184, 79, 3, 0], [424, 3531, 8758, 279, 1566, 47, 14835, 23218, 242, 7014, 2466, 1222, 79, 5, 5525, 49470, 90, 75, 5525, 36823, 853, 42, 53771, 76, 3818, 127, 2, 9, 8758, 9739, 12746, 3183, 815, 5189, 2573, 184, 47, 21504, 3926, 12, 605, 8758, 307, 3, 739, 3, 8397, 4419, 365, 13649, 750, 5659, 724, 11, 3, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[150, 11717, 440, 5814, 326, 424,

In [26]:
tokenized_datasets = raw_dataset.map(preprocess_function, batched=True)

Loading cached processed dataset at /Users/michael/.cache/huggingface/datasets/json/default-1d10f823a78c2103/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51/cache-5df8d0e1aa39b6be.arrow
Loading cached processed dataset at /Users/michael/.cache/huggingface/datasets/json/default-1d10f823a78c2103/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51/cache-43f434706d963a93.arrow
Loading cached processed dataset at /Users/michael/.cache/huggingface/datasets/json/default-1d10f823a78c2103/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51/cache-1f435a57217cd7e8.arrow


In [27]:
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq

model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Metal device set to: Apple M1 Pro


2023-02-14 17:08:41.681583: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-14 17:08:41.681997: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-en-ROMANCE.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [28]:
batch_size = 4096 #was 10 before
learning_rate = 1 #2e-5
weight_decay = 0.0001 #?
num_train_epochs = 1 #1 before

model_name = model_checkpoint.split("/")[-1]
push_to_hub_model_id = f"{model_name}-finetuned-{source_lang}-to-{target_lang}"

In [29]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="np")

generation_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="np", pad_to_multiple_of=128)

In [30]:
!pip install datasets
train_dataset = model.prepare_tf_dataset(
    tokenized_datasets["train"],
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator,
)

validation_dataset = model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    batch_size=batch_size,
    shuffle=False,
    collate_fn=data_collator,
)

generation_dataset = model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    batch_size=8,
    shuffle=False,
    collate_fn=generation_data_collator,
)

In [31]:
from transformers import AdamWeightDecay
import tensorflow as tf

optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [32]:
from transformers.keras_callbacks import KerasMetricCallback
import numpy as np


def metric_fn(eval_predictions):
    preds, labels = eval_predictions
    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds
    ]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # We use -100 to mask labels - replace it with the tokenizer pad token when decoding
    # so that no output is emitted for these
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}
    result["gen_len"] = np.mean(prediction_lens)
    return result


metric_callback = KerasMetricCallback(
    metric_fn=metric_fn, eval_dataset=generation_dataset, predict_with_generate=True, use_xla_generation=True, 
    generate_kwargs={"max_length": 128}
)

In [ ]:
from transformers.keras_callbacks import PushToHubCallback
from tensorflow.keras.callbacks import TensorBoard

tensorboard_callback = TensorBoard(log_dir="./translation_model_save/logs")

push_to_hub_callback = PushToHubCallback(
    output_dir="./translation_model_save",
    tokenizer=tokenizer,
    hub_model_id=push_to_hub_model_id,
)

callbacks = [metric_callback, tensorboard_callback, push_to_hub_callback]

model.fit(
    train_dataset, validation_data=validation_dataset, epochs=1000, callbacks=callbacks
)

Cloning https://huggingface.co/mpham8/opus-mt-en-ROMANCE-finetuned-bam-to-fr into local empty directory.


Download file tf_model.h5:   0%|          | 7.43k/298M [00:00<?, ?B/s]

Download file target.spm:   0%|          | 1.44k/780k [00:00<?, ?B/s]

Download file logs/train/events.out.tfevents.1676343352.7e3b0514c9b2.22042.2.v2:   0%|          | 8.00k/3.88M …

Download file logs/train/events.out.tfevents.1676338414.7e3b0514c9b2.6402.0.v2:   0%|          | 16.5k/3.82M […

...

Clean file logs/train/events.out.tfevents.1676345832.7e3b0514c9b2.22042.4.v2:   0%|          | 1.00k/3.88M [00…

Clean file logs/train/events.out.tfevents.1676345957.7e3b0514c9b2.39977.0.v2:   0%|          | 1.00k/3.82M [00…

Clean file tf_model.h5:   0%|          | 1.00k/298M [00:00<?, ?B/s]

Epoch 1/1000


2023-02-14 17:11:15.497634: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-02-14 17:11:27.052249: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [ ]:
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM

# You can of course substitute your own username and model here if you've trained and uploaded it!
model_name = 'mpham8/opus-mt-en-ROMANCE-finetuned-bam-to-fr'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_name)

In [ ]:
input_text  = "Farigan t'a la."
#if 't5' in model_name: 
    #input_text = "translate English to Romanian: " + input_text
tokenized = tokenizer([input_text], return_tensors='np')
out = model.generate(**tokenized, max_length=128)
print(out)

In [ ]:
with tokenizer.as_target_tokenizer():
    print(tokenizer.decode(out[0], skip_special_tokens=True))